In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import babel.dates
from babel import Locale

### Кейс по нахождению наиболее близкой даты звонка при сопоставлении двух выгрузок из разных программ, а также по выявлению звонков нарушающих ФЗ-230 (телефонные звонки могут осуществляться не более одного раза в день, двух раз в неделю и восьми раз в месяц)

## Алтухов Николай 

Пояснение: есть выгрузка из программы, данные в которой создаются автоматически этой программой (при начале вызова сразу проставляется номер телефона и дата вызова). И есть вторая выгрузка, уже из другой программы, данные в которой заполняются вручную сотрудником; плюс у сотрудника может быть другой часовой пояс; соответственно, даты вызова будут отличаться от тех, которые выставляются программой автоматически.

Задача - сопоставить эти две выгрузки, путем присоединения данных по наиболее близкой дате, с целью вытягивания дополнительной информации (например, столбцов с результатом взаимодействия по звонку или статусом договора) из нужной выгрузки
________

Генерация 10 тысяч случайных российских номеров телефона

In [2]:
phone_numbers = []
for _ in range(10000):
    number = "89" + ''.join(random.choice('0123456789') for _ in range(9))
    phone_numbers.append(number)
phone_numbers = np.array(phone_numbers)

Генерация количества звонков по каждому номеру и дат вызова

In [3]:
# Используем распределение Пуассона для генерации случайного количество звонков на номера телефонов
quantities = np.random.poisson(5, 10000)

# Замена всех нулей на единицы, чтоб все 10000 телефонов остались
quantities[quantities == 0] = 1

# Повторяем каждый элемент массива phone_numbers нужное количество раз
phone_numbers_repeated = np.repeat(phone_numbers, quantities)

In [4]:
start_date = datetime(2024, 7, 1, 0, 0, 0)
end_date = datetime(2024, 7, 31, 23, 59, 59)
delta = end_date - start_date
total_sec = int(delta.total_seconds())
random_dates = []
for _ in range(len(phone_numbers_repeated)):
    # выбор случайной секунды в июле 2024
    random_second = random.randint(0, total_sec)
    random_date = start_date + timedelta(seconds=random_second)
    random_dates.append(random_date)

Генерация результатов взаимодействия, которые мы будем подтягивать во второй датафрейм

In [5]:
results = [
    "Платеж подтвержден",
    "Ответ третьих лиц",
    "Должник отказался платить",
    "Просит реструктуризацию",
    "Должник попросил перезвонить позже"
]

random_results = random.choices(results, k=len(random_dates))


In [6]:
df_calls = pd.DataFrame({'Номер телефона': phone_numbers_repeated, 'Дата вызова': random_dates, "Результат взаимодействия":random_results})

In [7]:
df_calls

,Номер телефона,Дата вызова,Результат взаимодействия
0,89911356051,2024-07-01 23:42:19,Ответ третьих лиц
1,89911356051,2024-07-30 10:20:03,Должник попросил перезвонить позже
2,89911356051,2024-07-08 09:31:03,Должник попросил перезвонить позже
3,89927551157,2024-07-15 11:08:40,Ответ третьих лиц
4,89927551157,2024-07-13 19:41:17,Должник попросил перезвонить позже
...,...,...,...
50233,89464456197,2024-07-14 13:03:51,Ответ третьих лиц
50234,89464456197,2024-07-10 11:09:42,Просит реструктуризацию
50235,89464456197,2024-07-02 09:59:18,Ответ третьих лиц
50236,89464456197,2024-07-14 01:31:12,Платеж подтвержден


Добавление номера недели и дня недели, для дальнейшего анализа по выявлению звонков нарушающих ФЗ-230

In [8]:
# Устанавливаем локаль для русского языка
locale = Locale('ru', 'RU')

# Функция для получения названия дня недели на русском языке
def get_day_name(date, locale):
    return babel.dates.format_datetime(date, 'EEEE', locale=locale)

df_calls['Номер недели'] = df_calls['Дата вызова'].dt.isocalendar().week
df_calls['День недели'] = df_calls['Дата вызова'].apply(lambda x: get_day_name(x, locale))

df_calls

,Номер телефона,Дата вызова,Результат взаимодействия,Номер недели,День недели
0,89911356051,2024-07-01 23:42:19,Ответ третьих лиц,27,понедельник
1,89911356051,2024-07-30 10:20:03,Должник попросил перезвонить позже,31,вторник
2,89911356051,2024-07-08 09:31:03,Должник попросил перезвонить позже,28,понедельник
3,89927551157,2024-07-15 11:08:40,Ответ третьих лиц,29,понедельник
4,89927551157,2024-07-13 19:41:17,Должник попросил перезвонить позже,28,суббота
...,...,...,...,...,...
50233,89464456197,2024-07-14 13:03:51,Ответ третьих лиц,28,воскресенье
50234,89464456197,2024-07-10 11:09:42,Просит реструктуризацию,28,среда
50235,89464456197,2024-07-02 09:59:18,Ответ третьих лиц,27,вторник
50236,89464456197,2024-07-14 01:31:12,Платеж подтвержден,28,воскресенье


Теперь нужно сгенерировать выгрузку из программы, которой пользуется сотрудник. У него может быть другой часовой пояс, и проставить дату вызова он может в течение некоторого времени с начала звонка.

Для этого я возьму созданный датафрейм и изменю даты вызова. Даты будут отличаться на случайное число в недискретном диапазоне 0-3 часа, а также у некоторых (10% от общего количества) будет изменен часовой пояс (я буду добавлять +4, -4 часа)

В выгрузке из программы сотрудника будет 5 тысяч звонков, к которым нужно подтянуть данные по Результату взаимодействия

In [9]:
df_to_match = df_calls[['Номер телефона','Дата вызова']].sample(5000).copy()

In [10]:
random_hours = np.random.uniform(0, 3, size=len(df_to_match))
random_timezone = random.choices([0,4,-4], weights=[90, 5, 5], k=len(df_to_match))

random_hours_timedelta = pd.to_timedelta(random_hours, unit='h')
random_timezone_timedelta = pd.to_timedelta(random_timezone, unit='h')
# Применение случайных значений к датам
df_to_match['Дата вызова от сотрудника'] = df_to_match['Дата вызова'] + random_hours_timedelta + random_timezone_timedelta
df_to_match['Дата вызова от сотрудника'] = df_to_match['Дата вызова от сотрудника'].dt.round('s')
df_to_match = df_to_match.rename(columns={'Дата вызова':"old.Дата вызова"})

In [11]:
df_to_match

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника
15048,89231412205,2024-07-17 14:05:01,2024-07-17 15:02:28
23941,89872457440,2024-07-04 14:54:04,2024-07-04 16:37:38
30433,89648032886,2024-07-20 19:03:43,2024-07-20 19:52:56
33647,89844019556,2024-07-15 10:31:51,2024-07-15 11:55:15
15242,89700628200,2024-07-21 16:25:42,2024-07-21 21:44:16
...,...,...,...
34358,89681461663,2024-07-03 12:20:33,2024-07-03 09:20:46
40031,89099353036,2024-07-16 14:06:48,2024-07-16 16:10:59
48192,89308159532,2024-07-13 10:39:19,2024-07-13 12:03:01
49976,89907488361,2024-07-08 06:18:52,2024-07-08 06:48:58


In [12]:
df_to_match = df_to_match.sort_values(by=['Номер телефона', 'Дата вызова от сотрудника'])

Перемешаем строки выгрузки звонков первого датафрейма, чтоб немного усложнить задачу

In [13]:
df_calls_shuffled = df_calls.sample(frac=1).reset_index(drop=True)

Объединим датафрейм выгрузки сотрудника с перемешанным первым датафреймом, путем соединения LEFT JOIN

In [14]:
merged_df = df_to_match.merge(df_calls_shuffled, on='Номер телефона', how='left')

Вычисление разницы между датами

In [15]:
merged_df

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели
0,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-05 07:28:58,Просит реструктуризацию,27,пятница
1,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-21 10:28:07,Должник отказался платить,29,воскресенье
2,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-22 07:28:59,Просит реструктуризацию,30,понедельник
3,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-29 08:33:46,Должник отказался платить,31,понедельник
4,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-06 07:22:59,Ответ третьих лиц,27,суббота
...,...,...,...,...,...,...,...
29938,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-11 08:27:23,Должник попросил перезвонить позже,28,четверг
29939,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-11 18:30:26,Должник попросил перезвонить позже,28,четверг
29940,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-05 14:41:26,Просит реструктуризацию,27,пятница
29941,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-25 03:20:17,Просит реструктуризацию,30,четверг


In [16]:
merged_df['date_diff'] = abs(merged_df['Дата вызова от сотрудника'] - merged_df['Дата вызова'])

In [17]:
merged_df

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
0,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-05 07:28:58,Просит реструктуризацию,27,пятница,5 days 04:38:09
1,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-21 10:28:07,Должник отказался платить,29,воскресенье,10 days 22:21:00
2,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-22 07:28:59,Просит реструктуризацию,30,понедельник,11 days 19:21:52
3,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-29 08:33:46,Должник отказался платить,31,понедельник,18 days 20:26:39
4,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-06 07:22:59,Ответ третьих лиц,27,суббота,4 days 04:44:08
...,...,...,...,...,...,...,...,...
29938,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-11 08:27:23,Должник попросил перезвонить позже,28,четверг,5 days 15:47:42
29939,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-11 18:30:26,Должник попросил перезвонить позже,28,четверг,6 days 01:50:45
29940,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-05 14:41:26,Просит реструктуризацию,27,пятница,0 days 01:58:15
29941,89999953312,2024-07-05 14:41:26,2024-07-05 16:39:41,2024-07-25 03:20:17,Просит реструктуризацию,30,четверг,19 days 10:40:36


Нахождение строки с минимальной разницей для каждой комбинации "Номер телефона" и "Дата вызова от сотрудника"

In [18]:
result_df = merged_df.loc[merged_df.groupby(['Номер телефона', 'Дата вызова от сотрудника'])['date_diff'].idxmin()]

In [19]:
result_df

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
5,89000505484,2024-07-10 10:04:28,2024-07-10 12:07:07,2024-07-10 10:04:28,Должник отказался платить,28,среда,0 days 02:02:39
17,89000505484,2024-07-11 03:25:36,2024-07-11 02:21:16,2024-07-11 03:25:36,Просит реструктуризацию,28,четверг,0 days 01:04:20
28,89000621072,2024-07-03 16:13:06,2024-07-03 19:06:26,2024-07-03 16:13:06,Должник попросил перезвонить позже,27,среда,0 days 02:53:20
29,89000714834,2024-07-04 13:51:16,2024-07-04 15:45:33,2024-07-04 13:51:16,Должник отказался платить,27,четверг,0 days 01:54:17
35,89000851259,2024-07-02 18:15:15,2024-07-02 18:53:36,2024-07-02 18:15:15,Должник отказался платить,27,вторник,0 days 00:38:21
...,...,...,...,...,...,...,...,...
29914,89999342329,2024-07-18 09:14:34,2024-07-18 11:41:30,2024-07-18 09:14:34,Ответ третьих лиц,29,четверг,0 days 02:26:56
29923,89999695169,2024-07-19 19:29:44,2024-07-19 17:30:09,2024-07-19 19:29:44,Ответ третьих лиц,29,пятница,0 days 01:59:35
29927,89999695169,2024-07-31 16:35:43,2024-07-31 15:20:03,2024-07-31 16:35:43,Платеж подтвержден,31,среда,0 days 01:15:40
29931,89999779602,2024-07-16 23:30:34,2024-07-16 23:53:45,2024-07-16 23:30:34,Ответ третьих лиц,29,вторник,0 days 00:23:11


На основе столбца "old.Дата вызова" сверим полученный результат

In [20]:
result_df[result_df['old.Дата вызова']!=result_df['Дата вызова']]

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
117,89005333496,2024-07-10 07:35:54,2024-07-10 08:50:40,2024-07-10 08:53:39,Должник попросил перезвонить позже,28,среда,0 days 00:02:59
268,89007792495,2024-07-13 09:56:19,2024-07-13 10:52:08,2024-07-13 11:15:56,Просит реструктуризацию,28,суббота,0 days 00:23:48
648,89021501991,2024-07-17 18:15:38,2024-07-17 19:49:56,2024-07-17 21:20:50,Платеж подтвержден,29,среда,0 days 01:30:54
667,89021501991,2024-07-18 23:06:15,2024-07-19 01:44:04,2024-07-19 02:32:17,Должник попросил перезвонить позже,29,пятница,0 days 00:48:13
833,89025916366,2024-07-11 03:50:17,2024-07-11 00:00:19,2024-07-10 23:11:21,Ответ третьих лиц,28,среда,0 days 00:48:58
...,...,...,...,...,...,...,...,...
29038,89968214942,2024-07-05 08:10:40,2024-07-05 14:22:31,2024-07-05 11:40:37,Должник отказался платить,27,пятница,0 days 02:41:54
29200,89974676895,2024-07-05 01:53:15,2024-07-04 22:23:21,2024-07-04 20:27:14,Должник отказался платить,27,четверг,0 days 01:56:07
29269,89978629660,2024-07-14 19:13:35,2024-07-14 15:48:14,2024-07-14 18:58:16,Должник отказался платить,28,воскресенье,0 days 03:10:02
29842,89997192668,2024-07-14 13:21:17,2024-07-14 16:10:39,2024-07-14 15:02:03,Просит реструктуризацию,28,воскресенье,0 days 01:08:36


106 значений подтянулись неверно. Проверим по какой причине это произошло, взяв один экземпляр для проверки

In [21]:
merged_df[merged_df['Номер телефона']==result_df[result_df['old.Дата вызова']!=result_df['Дата вызова']].iloc[0,0]]

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
117,89005333496,2024-07-10 07:35:54,2024-07-10 08:50:40,2024-07-10 08:53:39,Должник попросил перезвонить позже,28,среда,0 days 00:02:59
118,89005333496,2024-07-10 07:35:54,2024-07-10 08:50:40,2024-07-10 07:35:54,Платеж подтвержден,28,среда,0 days 01:14:46
119,89005333496,2024-07-10 07:35:54,2024-07-10 08:50:40,2024-07-29 05:21:25,Должник попросил перезвонить позже,31,понедельник,18 days 20:30:45


Видно, что "Дата вызова от сотрудника" находится ближе к "Дате вызова", чем "old.Дата вызова". Поэтому код сработал верно. Здесь влияние оказал часовой пояс. Обычно в таких выгрузках присутствует столбец с ФИО сотрудника, в таком случае можно из распределения специалистов взять регион, где работает сотрудник, и на основе этого региона вычислить часовой пояс.

Таким образом, удалось сопоставить наиближайшие даты из двух выгрузок

## Выявление звонков, нарушающих ФЗ-230

Напомню, ФЗ-230 дает ограничение для организаций: "телефонные звонки могут осуществляться не более одного раза в день, двух раз в неделю и восьми раз в месяц"

In [22]:
df_calls = df_calls.sort_values(by=['Номер телефона', 'Дата вызова'])

In [23]:
df_calls['Порядковый номер звонка за месяц'] = df_calls.groupby(['Номер телефона'])['Дата вызова'].rank(method='dense').astype(int)
df_calls['Порядковый номер звонка за неделю'] = df_calls.groupby(['Номер телефона', 'Номер недели'])['Дата вызова'].rank(method='dense').astype(int)
df_calls['Порядковый номер звонка за день'] = df_calls.groupby(['Номер телефона', 'Номер недели', 'День недели'])['Дата вызова'].rank(method='dense').astype(int)


In [24]:
df_calls

,Номер телефона,Дата вызова,Результат взаимодействия,Номер недели,День недели,Порядковый номер звонка за месяц,Порядковый номер звонка за неделю,Порядковый номер звонка за день
38994,89000093821,2024-07-19 18:29:19,Ответ третьих лиц,29,пятница,1,1,1
38996,89000093821,2024-07-21 08:10:01,Должник попросил перезвонить позже,29,воскресенье,2,2,1
38995,89000093821,2024-07-25 10:57:27,Ответ третьих лиц,30,четверг,3,1,1
38993,89000093821,2024-07-27 03:17:38,Должник отказался платить,30,суббота,4,2,1
11573,89000095013,2024-07-01 04:19:19,Ответ третьих лиц,27,понедельник,1,1,1
...,...,...,...,...,...,...,...,...
28659,89999953312,2024-07-11 18:30:26,Должник попросил перезвонить позже,28,четверг,4,2,2
28656,89999953312,2024-07-14 01:49:48,Должник отказался платить,28,воскресенье,5,3,1
28655,89999953312,2024-07-25 03:20:17,Просит реструктуризацию,30,четверг,6,1,1
28652,89999953312,2024-07-29 17:43:30,Должник попросил перезвонить позже,31,понедельник,7,1,1


Номера телефонов (без дубликатов), по которым было нарушение ФЗ-230 по количеству звонков за месяц:

In [25]:
pd.DataFrame({'Больше 8 звонков за месяц': df_calls[df_calls['Порядковый номер звонка за месяц']>8]['Номер телефона'].unique()})

,Больше 8 звонков за месяц
0,89000240923
1,89000505484
2,89003412163
3,89005074503
4,89006196146
...,...
724,89995878913
725,89996300790
726,89997192668
727,89998212911


Номера телефонов (без дубликатов), по которым было нарушение ФЗ-230 по количеству звонков за неделю:

In [26]:
pd.DataFrame({'Больше 2 звонков за неделю': df_calls[df_calls['Порядковый номер звонка за неделю']>2]['Номер телефона'].unique()})

,Больше 2 звонков за неделю
0,89000240923
1,89000393344
2,89000505484
3,89000621072
4,89000714834
...,...
3732,89998977056
3733,89999091846
3734,89999342329
3735,89999778544


Номера телефонов (без дубликатов), по которым было нарушение ФЗ-230 по количеству звонков за день:

In [27]:
pd.DataFrame({'Больше 1 звонка за день': df_calls[df_calls['Порядковый номер звонка за день']>1]['Номер телефона'].unique()})

,Больше 1 звонка за день
0,89000240923
1,89000393344
2,89000505484
3,89000621072
4,89001197608
...,...
3030,89998887057
3031,89998977056
3032,89999159081
3033,89999778544


Номера телефонов (без дубликатов), по которым было любое нарушение ФЗ-230:

In [28]:
dfs_list = [pd.DataFrame({'Больше 8 звонков за месяц': df_calls[df_calls['Порядковый номер звонка за месяц']>8]['Номер телефона'].unique()}).rename(columns={'Больше 8 звонков за месяц':"Номер с нарушением ФЗ-230"}),
            pd.DataFrame({'Больше 2 звонков за неделю': df_calls[df_calls['Порядковый номер звонка за неделю']>2]['Номер телефона'].unique()}).rename(columns={'Больше 2 звонков за неделю':"Номер с нарушением ФЗ-230"}),
            pd.DataFrame({'Больше 1 звонка за день': df_calls[df_calls['Порядковый номер звонка за день']>1]['Номер телефона'].unique()}).rename(columns={'Больше 1 звонка за день':"Номер с нарушением ФЗ-230"})
           ]
pd.DataFrame({'Номер с нарушением ФЗ-230': pd.concat(dfs_list,axis=0)["Номер с нарушением ФЗ-230"].unique()})

,Номер с нарушением ФЗ-230
0,89000240923
1,89000505484
2,89003412163
3,89005074503
4,89006196146
...,...
4606,89994716792
4607,89996196660
4608,89997748286
4609,89997860523


In [29]:
print(f'Напоминаю, общее количество номеров равно: {len(df_calls['Номер телефона'].unique())}')

Напоминаю, общее количество номеров равно: 10000


Таким образом, по 46% номеров совершается нарушение ФЗ-230. Можно предположить, что в таком случае программа-обзвонщик не имеет ограничения на количество звонков или работает с ошибками, поэтому некорректно выполняет эту функцию. Это необходимо исправлять.